# **Файнтюнинг модели BART-BASE для суммаризации диалогов.**

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

!pip install -q awscli
!pip install --upgrade datasets -q
!pip uninstall wandb -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 36.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0+cu121 requires torch==2.5.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.20.0+cu121 requi

In [2]:
# Импорт необходимых библиотек
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, GenerationConfig, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import random
from google.colab import drive

# Подключение Google Drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [3]:
# Загрузка набора данных
dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(dataset_name)
dataset

# Загрузка модели и токенизатора
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/bart-base-cnn")
model = BartForConditionalGeneration.from_pretrained("ainize/bart-base-cnn")

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

## **Файнтюнинг**

### **Предобработка набора данных**

In [4]:
# Функция для токенизации данных
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    max_length = 512

    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").input_ids

    return example

# Токенизация датасета
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

# Подвыборка датасета
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

### **Дообучение модели на предобработанном наборе данных**

In [5]:
# Параметры обучения
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to=None,
)

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [6]:
# Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.176000,0.124349
2,0.126400,0.098459


TrainOutput(global_step=64, training_loss=0.16884600929915905, metrics={'train_runtime': 2853.0633, 'train_samples_per_second': 0.088, 'train_steps_per_second': 0.022, 'total_flos': 76217057280000.0, 'train_loss': 0.16884600929915905, 'epoch': 2.0})

In [7]:
# Перемещение модели на устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### **Оценка качества модели при помощи метрики ROUGE**

In [8]:
# Оценка модели с использованием метрики ROUGE
rouge = evaluate.load('rouge')

# Получаем тестовые данные: 10 диалогов и соответствующих им аннотаций
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Список для хранения аннотаций, сгенерированных моделью
model_summaries = []

for dialogue in dialogues:
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    # Токенизация и перемещение на устройство
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Генерация выходных данных модели
    model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))

    # Декодирование выходных данных
    model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    model_summaries.append(model_text_output)

# Вычисляем метрики ROUGE
model_results = rouge.compute(
    predictions=model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

print('FINE-TUNED MODEL:')
print(model_results)

FINE-TUNED MODEL:
{'rouge1': 0.3482452334545779, 'rouge2': 0.1188431731820692, 'rougeL': 0.2994675542103052, 'rougeLsum': 0.30022403196800695}


In [9]:
# Сохранение модели в Google Drive
model_path = "/content/drive/MyDrive/bartbase"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/bartbase/tokenizer_config.json',
 '/content/drive/MyDrive/bartbase/special_tokens_map.json',
 '/content/drive/MyDrive/bartbase/tokenizer.json')